<p style="font-family: Arial; font-size:3.75vw;color:purple; font-style:bold"><br>
pandas Exercise Notebook Solutions
</p><br>

# Exercise Notebook Instructions

### 1. Important: Only modify the cells which instruct you to modify them - leave "do not modify" cells alone.  

The code which tests your responses assumes you have run the startup/read-only code exactly.

### 2. Work through the notebook in order.

Some of the steps depend on previous, so you'll want to move through the notebook in order.

### 3. It is okay to use libraries.

You may find some questions are fairly straightforward to answer using built-in library functions.  That's totally okay - part of the point of these exercises is to familiarize you with the commonly used functions.

### 4. Seek help if stuck

If you get stuck, don't worry!  You can either review the videos/notebooks from this week, ask in the course forums, or look to the solutions for the correct answer.  BUT, be careful about looking to the solutions too quickly.  Struggling to get the right answer is an important part of the learning process.

In [1]:
import pandas as pd
import numpy as np
import os.path

In this exercise notebook you will have the opportunity to load the MovieLens database and perform additional analysis.

First let's load the data into a Pandas Dataframe:

In [2]:
# DO NOT MODIFY

# set here the relative path to the movielens folder
MOVIELENS="../../../movielens"

movies = pd.read_csv(os.path.join(MOVIELENS, 'movies.csv'), sep=',')
ratings = pd.read_csv(os.path.join(MOVIELENS, 'ratings.csv'), sep=',')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


## Exercise 1: Find the Minimum Rating

Let's start by computing the minimum rating.

In the next cell, define the `min_rating` variable to be the minimum rating across all of the DataFrame:

In [5]:
min_rating = None
### BEGIN SOLUTION
min_rating = ratings.rating.min()
### END SOLUTION

In [6]:
# DO NOT MODIFY

assert isinstance(min_rating, np.float64), "Try again, make sure you are taking the min of just 1 column"
assert abs(min_rating - 0.5) < .01, "Try again, the minimum should be 0.5"

## Exercise 2: Find the Mean Rating of a Movie

The movie **Toy Story** has `movieId` 1. Find out the mean rating for Toy Story. For this exercise you just need to use the `ratings` DataFrame:


In [7]:
toy_story_rating = None
### BEGIN SOLUTION
toy_story_rating = ratings[ratings.movieId == 1].rating.mean()
### END SOLUTION

In [8]:
assert abs(toy_story_rating - 3.92) < 0.01, "Try again, select only the rows where the movieId is equal to 1"

## Exercise 3: Find the Most Common Rating

Next, find which rating is the most common.

In [9]:
rating_counts = None
### BEGIN SOLUTION
rating_counts = ratings.rating.value_counts()
### END SOLUTION

In [10]:
# DO NOT MODIFY

assert rating_counts.iloc[:1].index[0] == 4., "Print out rating_counts and try to understand what is wrong"

It is also interesting to inspect step by step what the statement above is doing:

    rating_counts.iloc[:1].index[0]

print all the intermediate stages of this expression, read the documentation of the functions used here.

## Exercise 4: Usage of the Index in pandas

In `numpy` the way to point to a specific entry in an array is by using its integer position. In `pandas` you can do the same with `iloc`, but you also have the option of defining a column in a DataFrame as an `index` and refer to rows using `index` labels instead of integer locations using `loc`.

For example in `rating_counts` defined above, the rating is the `index`, so we can reference a value either by its position or by its rating.

For example we can reference the 4th record either with `iloc` and its position or with `loc` and its label:

In [11]:
rating_counts.iloc[3]

2200156

In [12]:
rating_counts.loc[3.5]

2200156

## Exercise 5: Set an Index for the Movies DataFrame

Movies have one column `movieId` that is a natural way of uniquely identifying each row, when that is the case, it is useful to turn that into an index.

In [13]:
movies = movies.set_index("movieId")

In [14]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


Ratings instead do not have a row identifier, both `userId` and `movieId` reference records in other dataframes, therefore there is no good candidate for an index so we can just leave the default integer indexing.

## Exercise 6: Year with Maximum Standard Deviation in the Rating

First assignment is to find which year has the maximum standard deviation in the rating, **not** the maximum value of the standard deviation, but the year when it occurs.
You can use the `idxmax` method, look at its documentation on the pandas website, https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.idxmax.html

First we want to convert the timestamp into a datetime object:

In [15]:
# DO NOT MODIFY

ratings['parsed_time'] = pd.to_datetime(ratings['timestamp'], unit='s')

We can then access to datetime related fields through `dt`, for example:

In [16]:
ratings.parsed_time.dt.month.head()

0    4
1    4
2    4
3    4
4    4
Name: parsed_time, dtype: int64

In [17]:
def find_year_with_max_std(ratings):
    """Function to find the year with the larger standard deviation in rating"""
    ### BEGIN SOLUTION
    return ratings.rating.groupby(ratings.parsed_time.dt.year).std().idxmax()
    ### END SOLUTION

In [18]:
assert find_year_with_max_std(ratings) == 1998, "Wrong year identified, try again!"

## Advanced Exercise 1: Identify Popular Movies

*In the rest of the notebook, we will introduce new concepts not covered in class, this will challenge you to read additional pandas documentation.*

First we would like only to consider movies that have a significant number of ratings. This task is complicated by the fact that movies and ratings are in 2 different DataFrames and we want to filter the `movies` DataFrame based on a statistics on the `ratings` DataFrame.

First let's compute the number of ratings per movie:

In [19]:
number_of_ratings = ratings.movieId.value_counts()

In [20]:
number_of_ratings.head()

296    67310
356    66172
318    63366
593    63299
480    59715
Name: movieId, dtype: int64

Now we want to filter this pandas Series object and keep only the rows where the count is larger or equal to 100.
We don't want to pollute our analysis with movies with a tiny number of reviews:

In [21]:
number_of_ratings_of_popular_movies = None
### BEGIN SOLUTION
number_of_ratings_of_popular_movies = number_of_ratings[number_of_ratings >= 100]
### END SOLUTION

In [22]:
assert len(number_of_ratings_of_popular_movies) == 8546, "Try again, check that movies with 100 ratings are accepted"

Finally we want to use the `reindex` function to change the index of movies, this will create a new DataFrame with a new index that contains the movieId of only the most popular movies.

The value of all the rows of `movies` that have the same movieId will be copied over to the new `popular_movies` dataset, the rest will be discarded.

In [23]:
all_popular_movies = movies.reindex(number_of_ratings_of_popular_movies)

## Advanced Exercise 2: Data Cleaning

Everytime we perform a reindexing operation, `pandas` will create a row for every value of the new index, even if it doesn't exist in the original data structure, and it will mark those data as missing with `NaN` (Not A Number).

Always check if reindexing generated invalid data:

In [24]:
all_popular_movies.isnull().sum()

title     699
genres    699
dtype: int64

In the next cell we want to drop the invalid data, look for a `pandas` function that performs that operation (it starts with "drop"!).

In [25]:
popular_movies = None
### BEGIN SOLUTION
popular_movies = all_popular_movies.dropna()
### END SOLUTION

In [26]:
assert len(popular_movies) == 7847, "Try again, check the documentation of the function you used"

## Advanced Exercise 3: Filter by Genre

Let's implement a general function that filters movies by genres:

In [27]:
def filter_by_genre(input_movies, genre):
    """Return only movies of a specific genre"""
    ### BEGIN SOLUTION
    return input_movies[input_movies.genres.str.contains(genre)]
    ### END SOLUTION

Then let's apply it to the `popular_movies` dataset to retain only the "Fantasy" movies:

In [28]:
fantasy_movies = filter_by_genre(popular_movies, "Fantasy")

In [29]:
assert len(fantasy_movies) == 382, """Try again, Make sure you are filtering the popular movies"""

## Advanced Exercise 4: Join Movies and Ratings

Let's create a single `DataFrame` that contains both titles and mean ratings of the popular fantasy movies.

Titles are only available in the `movies` `DataFrame`, while ratings in the `ratings` `DataFrame`, we would like to create a single DataFrame that includes Title.

Create the `mean_ratings` variable by computing the mean rating for each movie:

In [30]:
mean_ratings = None
### BEGIN SOLUTION
mean_ratings = ratings.rating.groupby(ratings.movieId).mean()
### END SOLUTION

In this case we don't even need to use a join operation, we can just create a new column in the `fantasy_movies` DataFrame. This will automatically match the index of `mean_ratings` with the index of `fantasy_movies` and attach to each movie its rating. Ratings for movies that are not in the `fantasy_movies` DataFrame are discarded.

The recommended way of creating columns in the recent versions of `pandas` is through the `assign` function, read its documentation!

In [31]:
fantasy_movies.assign?

In [32]:
fantasy_movies_with_ratings = fantasy_movies.assign(rating = mean_ratings)

In [33]:
fantasy_movies_with_ratings.head()

,title,genres,rating
movieId,,,
8492,"Christmas Carol, A (Scrooge) (1951)",Drama|Fantasy,3.833778
8157,Jin Roh: The Wolf Brigade (Jin-Rô) (1998),Animation|Fantasy|Thriller,3.679928
8015,"Phantom Tollbooth, The (1970)",Adventure|Animation|Children|Fantasy,3.585903
7842,Dune (2000),Drama|Fantasy|Sci-Fi,3.568719
7657,Versus (2000),Action|Comedy|Fantasy|Horror,2.987013


In [34]:
assert fantasy_movies_with_ratings.loc[7842].title.startswith("Dune"), "Try again, missing or wrong title"

In [35]:
assert abs(fantasy_movies_with_ratings.loc[7842].rating - 3.56) < 0.01 , "Try again, missing or wrong rating"

## Advanced Exercise 5: Find the Highest Rated Fantasy Movie

Again we need to find the index where a column is max, in this case rating:

In [36]:
index_of_max_rating = None
### BEGIN SOLUTION
index_of_max_rating = fantasy_movies_with_ratings.rating.idxmax()
### END SOLUTION

In [37]:
highest_rated_fantasy_movie = fantasy_movies_with_ratings.loc[index_of_max_rating]

In [38]:
assert highest_rated_fantasy_movie.title.startswith("Princess"), "Try again"